<a href="https://colab.research.google.com/github/sfarrukhm/model_finetune/blob/main/modernbert_setfit_nli/training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU datasets transformers evaluate

In [2]:
from datasets import load_dataset
dataset=load_dataset('SetFit/mnli')
dataset=dataset.rename_columns({'text1':'premise','text2':'hypothesis','label':'labels'})

from collections import Counter

# Class distribution
train_labels = [sample['labels'] for sample in train_data]
print(f"Class distribution in train set: {Counter(train_labels)}")

Repo card metadata block was not found. Setting CardData to empty.


Class distribution in train set: Counter({2: 130903, 1: 130900, 0: 130899})


In [3]:
# Desired subset size and class-wise calculation
desired_subset_size = 30000
num_classes = 3
samples_per_class = desired_subset_size // num_classes

# Perform stratified sampling
def stratified_sampling(dataset, samples_per_class):
    # Group dataset by label
    class_labels = list(set(dataset['labels']))
    stratified_subset = []
    for label in class_labels:
        # Filter samples for the current class
        class_samples = dataset.filter(lambda x: x['labels'] == label)
        # Shuffle and select the required number of samples
        stratified_subset.extend(class_samples.shuffle(seed=42).select(range(samples_per_class)))
    return stratified_subset

# Apply stratified sampling
train_subset = stratified_sampling(train_data, samples_per_class)

# Convert the subset to Dataset
from datasets import Dataset
train_subset = Dataset.from_dict({key: [item[key] for item in train_subset] for key in train_subset[0].keys()})

# Verify the distribution
# from collections import Counter
# train_labels = train_subset['labels']
# print(f"Subset class distribution: {Counter(train_labels)}")
# print(f"Total samples in subset: {len(train_subset)}")


In [5]:
# Load the base model tokenizer
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint="answerdotai/ModernBERT-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_length=512
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
# def tokenizer_fn(example)
def tokenize_function(example):
  tokenized_example=tokenizer(example['premise'],example['hypothesis'],truncation=True,padding='max_length',return_tensors='pt')
  return tokenized_example

train_subset_tokenized=train_subset.map(tokenize_function,batched=True)
val_data_tokenized=val_data.map(tokenize_function,batched=True)
test_data_tokenized=test_data.map(tokenize_function,batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

####  Set the trainer

In [9]:
# load base model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# define training arguments
from transformers import TrainingArguments, Trainer
output_dir="/content/drive/MyDrive/huggingface/model_trained/modernbert-nli_setfit"
hub_model_id="modernbert-setfit-nli"


training_args=TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=3,
    save_strategy='epoch',
    evaluation_strategy='epoch'
    hub_model_id=hub_model_id,
)

# set the training loop
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset_tokenized,
    eval_dataset=val_data_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-31-1934bcb82e7d>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.467800,0.424898


Epoch,Training Loss,Validation Loss
1,0.467800,0.424898
2,0.316000,0.554999
3,0.179500,0.905628


TrainOutput(global_step=11250, training_loss=0.37069759623209636, metrics={'train_runtime': 2381.9295, 'train_samples_per_second': 37.784, 'train_steps_per_second': 4.723, 'total_flos': 4230516689574336.0, 'train_loss': 0.37069759623209636, 'epoch': 3.0})

In [ ]:
# Save the model and push to hub
trainer.save_model()
trainer.push_to_hub("Fine-tuned ModernBERT for NLI")